In [1]:
using JuAFEM
using Tensors
using TimerOutputs
using UnicodePlots
const to = TimerOutput();

## Homogenization

We consider the equation

$$\sigma \cdot \nabla = 0$$

together with the constraint

$$ \langle \epsilon[u] \rangle = \overline{\epsilon}$$ 

where $\overline{\epsilon}$ is prescribed and 

$$ \langle f \rangle = \frac{1}{|\Omega|}  \int_\Omega f d \Omega $$

In [2]:
GEOSHAPE = Tetrahedron
REFSHAPE = RefTetrahedron
QUADRATURE_ORDER = 2
DIM = 3

3

In [3]:
function compute_grid(n, f)
    dim = 3
    grid = generate_grid(GEOSHAPE, (10, 10, 10));
    # Extract the left boundary
    addnodeset!(grid, "clamped", f);
end

compute_grid (generic function with 1 method)

In [4]:
# Interpolations and values
function create_interpolations()
    interpolation_space = Lagrange{DIM, REFSHAPE, 1}()
    quadrature_rule = QuadratureRule{DIM, REFSHAPE}(QUADRATURE_ORDER)
    face_quadrature_rule = QuadratureRule{DIM-1, REFSHAPE}(QUADRATURE_ORDER)
    cellvalues = CellVectorValues(quadrature_rule, interpolation_space);
    facevalues = FaceVectorValues(face_quadrature_rule , interpolation_space);
    return cellvalues, facevalues
end

create_interpolations (generic function with 1 method)

In [5]:
function create_dofhandler(grid)
    # DofHandler
    dh = DofHandler(grid)
    push!(dh, :u, DIM) # Add a displacement field
    close!(dh)
    return dh
end

create_dofhandler (generic function with 1 method)

In [6]:
function create_boundaryconditions(grid, dh, f)
    # Boundaryconditions
    dbc = DirichletBoundaryConditions(dh)
    # Add a homogenoush boundary condition on the "clamped" edge
    add!(dbc, :u, getnodeset(grid, "clamped"), f, collect(1:DIM))
    close!(dbc)
    t = 0.0
    update!(dbc, t)
    return dbc
end

create_boundaryconditions (generic function with 1 method)

In [7]:
# Create the stiffness tensor
function create_stiffness(Emod = 200e9, ν = 0.3)
    λ = Emod*ν / ((1+ν) * (1 - 2ν))
    μ = Emod / (2(1+ν))
    δ(i,j) = i == j ? 1.0 : 0.0
    g(i,j,k,l) = λ*δ(i,j)*δ(k,l) + μ*(δ(i,k)*δ(j,l) + δ(i,l)*δ(j,k))
    E = SymmetricTensor{4, DIM}(g)
    return E
end

create_stiffness (generic function with 3 methods)

In [8]:
function doassemble3!{dim}(cellvalues::CellVectorValues{dim}, facevalues::FaceValues,
                           K::SparseMatrixCSC, grid::Grid, dh::DofHandler, E::SymmetricTensor{4, dim})

    f = zeros(ndofs(dh))
    C = zeros(ndofs(dh), 9)
    assembler = start_assemble(K, f)
    
    n_basefuncs = getnbasefunctions(cellvalues)
    global_dofs = zeros(Int, ndofs_per_cell(dh))

    Ke = zeros(n_basefuncs, n_basefuncs) # Local stiffness mastrix
    Ce = zeros(n_basefuncs, 9)
    Ω = 8

    δɛ = [zero(SymmetricTensor{2, dim}) for i in 1:n_basefuncs]
    ∇δu = [zero(Tensor{2, dim}) for i in 1:n_basefuncs]

    @inbounds for  cell in CellIterator(dh)
        fill!(Ke, 0)
        fill!(Ce, 0)
        reinit!(cellvalues, cell)
        for q_point in 1:getnquadpoints(cellvalues)
            for i in 1:n_basefuncs
                δɛ[i] = symmetric(shape_gradient(cellvalues, q_point, i))
                ∇δu[i] = shape_gradient(cellvalues, q_point, i)
            end
            dΩ = getdetJdV(cellvalues, q_point)
            for i in 1:n_basefuncs
                #Ce[i, :] += -1 / Ω * tovoigt(∇δu[i]) * dΩ
                δɛC = δɛ[i] ⊡ E
                for j in 1:n_basefuncs
                    Ke[i, j] += 1/Ω * (δɛC ⊡ δɛ[j]) * dΩ
                end
            end
        end
        
        for face in 1:nfaces(cell)
            if onboundary(cell, face)
                reinit!(facevalues, cell, face)
                for q_point in 1:getnquadpoints(facevalues)
                    dΓ = getdetJdV(facevalues, q_point)
                    n = getnormal(facevalues, q_point)
                    for i in 1:n_basefuncs
                        δu = shape_value(facevalues, q_point, i)
                        Ce[i, :] += -1 / Ω * tovoigt((δu ⊗ n) * dΓ)
                    end
                end
            end
        end
        
        celldofs!(global_dofs, cell)
        for (i, dof) in enumerate(global_dofs)
            C[dof, :] += Ce[i, :]
        end
        assemble!(assembler, Ke, global_dofs)
    end
    return f, C
end

doassemble3! (generic function with 1 method)

In [48]:
dbc_f = x -> (x ≈ Vec{3}((-1,-1,-1)) || x ≈ Vec{3}((1,1,1)))  ? true : false
grid = compute_grid(10, dbc_f)
dh = create_dofhandler(grid)

cellvalues, facevalues = create_interpolations()

# Boundary conditions
eps = basevec(Vec{DIM}, 1) ⊗ basevec(Vec{DIM}, 2)
ɛ_bar = 0.05 * (eps + eps')
dbc = create_boundaryconditions(grid, dh, (x,t) -> ɛ_bar ⋅ x)

K = create_sparsity_pattern(dh);
E = create_stiffness();

In [49]:
f, C = doassemble3!(cellvalues, facevalues, K, grid, dh, E)

([0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0  …  0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],
[0.000833333 0.0 … 0.0 0.0; 0.0 0.000833333 … 0.0 0.000833333; … ; 0.0 -0.00166667 … 0.0 -0.00166667; 0.0 0.0 … -0.00166667 0.0])

In [50]:
#apply!(K, f, dbc)
#uu = K \ f
fd = dbc.free_dofs
pd = dbc.dofs
up = dbc.values
Kff = K[fd, fd]
Kfp = K[fd, pd]
Cf = C[fd, :]
Cp = C[pd, :]

6×9 Array{Float64,2}:
 -0.00166667    0.0           0.0          …   0.0           0.0        
  0.0          -0.00166667    0.0              0.0          -0.00166667 
  0.0           0.0          -0.00166667      -0.00166667    0.0        
  0.000833333   0.0           0.0              0.0           0.0        
  0.0           0.000833333   0.0              0.0           0.000833333
  0.0           0.0           0.000833333  …   0.000833333   0.0        

In [51]:
lhs = [Kff       Cf
       Cf'   zeros(9,9)]
rhs = [-Kfp * up
       -tovoigt(ɛ_bar) - Cp' * up]

3996-element Array{Float64,1}:
 -8.01282e7
 -3.20513e7
 -1.60256e7
  0.0      
  0.0      
  0.0      
  0.0      
  0.0      
  0.0      
  0.0      
  0.0      
  0.0      
  0.0      
  ⋮        
  4.00641e7
 -8.01282e6
 -2.40385e7
  0.000125 
  0.000125 
 -0.0      
  0.000125 
  0.000125 
 -0.049875 
 -0.0      
 -0.0      
 -0.049875 

In [52]:
uf = lhs \ rhs
sigma = uf[end-8:end]
uu    = uf[1:end-9];
uu_full = zeros(ndofs(dh))
uu_full[dbc.dofs] = dbc.values
uu_full[dbc.free_dofs] = uu

3987-element Array{Float64,1}:
 -0.0501064  
 -0.041526   
 -0.000102204
 -0.0496548  
 -0.0320329  
 -0.000456636
 -0.0490984  
 -0.0222959  
 -0.000655487
 -0.0485562  
 -0.0124384  
 -0.000749581
 -0.0480384  
  ⋮          
  0.052559   
  0.00758191 
 -0.000841914
  0.0519921  
  0.0176831  
 -0.00077934 
  0.0513507  
  0.0279177  
 -0.000666551
  0.05059    
  0.0384232  
 -0.000477906

In [53]:
uu_full

3993-element Array{Float64,1}:
 -0.05       
 -0.05       
 -0.0        
 -0.0501064  
 -0.041526   
 -0.000102204
 -0.0496548  
 -0.0320329  
 -0.000456636
 -0.0490984  
 -0.0222959  
 -0.000655487
 -0.0485562  
  ⋮          
  0.0519921  
  0.0176831  
 -0.00077934 
  0.0513507  
  0.0279177  
 -0.000666551
  0.05059    
  0.0384232  
 -0.000477906
  0.05       
  0.05       
  0.0        

In [59]:
lhs = [K      C
       C' zeros(9,9)]
rhs   = [zeros(size(K,1)); -tovoigt(ɛ_bar)];
apply!(lhs, rhs, dbc)
u_σ   = lhs \ rhs;
sigma = u_σ[end-8:end]
uu    = u_σ[1:end-9];

#uu_full = zeros(ndofs(dh))
#uu_full[dbc.dofs] = dbc.values
#uu_full[dbc.free_dofs] = uu
σ_bar = fromvoigt(Tensor{2,3}, sigma)

3×3 Tensors.Tensor{2,3,Float64,9}:
  1.57981e6   7.67225e9   1.50949e6
  7.67225e9   1.57981e6   1.50949e6
 -1.37781e6  -1.37781e6  -1.40423e6

In [55]:
uu

3993-element Array{Float64,1}:
 -0.05       
 -0.05       
 -0.0        
 -0.0501064  
 -0.041526   
 -0.000102204
 -0.0496548  
 -0.0320329  
 -0.000456636
 -0.0490984  
 -0.0222959  
 -0.000655487
 -0.0485562  
  ⋮          
  0.0519921  
  0.0176831  
 -0.00077934 
  0.0513507  
  0.0279177  
 -0.000666551
  0.05059    
  0.0384232  
 -0.000477906
  0.05       
  0.05       
  0.0        

In [56]:
function check_residual2{dim}(cellvalues::CellVectorValues{dim}, 
                             σ_bar, ɛ_bar, u,  grid::Grid, dh::DofHandler, E::SymmetricTensor{4, dim})

    residual_u = zeros(ndofs(dh))
    residual_C = zeros(9)

    n_basefuncs = getnbasefunctions(cellvalues)
    global_dofs = zeros(Int, ndofs_per_cell(dh))

    residual_ue = zeros(n_basefuncs) # Local stiffness mastrix
    
    Ω = 8
    n_basefuncs = getnbasefunctions(cellvalues)
    global_dofs = zeros(Int, ndofs_per_cell(dh))

    ɛ = [zero(SymmetricTensor{2, dim}) for i in 1:n_basefuncs]
    for cell in CellIterator(dh)
        celldofs!(global_dofs, cell)
        uu = u[global_dofs]
        fill!(residual_ue, 0.0)
        compute_element_residual!(cellvalues, cell, residual_ue, residual_C, σ_bar, ɛ_bar, uu,  grid, dh, E)
        assemble!(residual_u, residual_ue, global_dofs)
    end
    return residual_u, residual_C
end


function compute_element_residual!{dim}(cellvalues::CellVectorValues{dim}, cell, residual_ue, residual_C,
                                  σ_bar, ɛ_bar, uz,  grid::Grid, dh::DofHandler, E::SymmetricTensor{4, dim})
    n_basefuncs = getnbasefunctions(cellvalues)
    ɛ = [zero(SymmetricTensor{2, dim}) for i in 1:n_basefuncs]
    uuv = reinterpret(Vec{3,Float64}, uz, (4,))
    #σv = fromvoigt(Tensor{2, dim}, σ_bar)
    Ω = 8
    reinit!(cellvalues, cell)
    for q_point in 1:getnquadpoints(cellvalues)
        ∇u_qp = function_gradient(cellvalues, q_point, uuv)
        ɛ_qp = symmetric(∇u_qp)
        dΩ = getdetJdV(cellvalues, q_point)
        residual_C .+= tovoigt(-1/Ω * (∇u_qp - ɛ_bar) * dΩ)
        for i in 1:n_basefuncs
            δ∇ui = shape_gradient(cellvalues, q_point, i)
            δɛi = symmetric(shape_gradient(cellvalues, q_point, i))
            residual_ue[i] += 1/Ω * (δɛi ⊡ E ⊡ ɛ_qp - σ_bar ⊡ δ∇ui) * dΩ
        end
    end
end

compute_element_residual! (generic function with 1 method)

In [57]:
@time res_u, res_c = check_residual2(cellvalues, σ_bar, ɛ_bar, uu, grid, dh, E)

  0.439975 seconds (553.75 k allocations: 22.442 MB, 21.08% gc time)


([7.62771e5,7.62771e5,-6.8088e5,-1316.51,-6.39355e6,1148.17,-1316.51,-6.39355e6,1148.17,-1316.51  …  1148.17,-1316.51,-6.39355e6,1148.17,-1316.51,-6.39355e6,1148.17,-7.62771e5,-7.62771e5,6.8088e5],[-2.53629e-7,-2.53629e-7,3.75131e-7,4.12799e-8,4.128e-8,0.000140261,1.67345e-7,1.67345e-7,0.000140261])

In [58]:
res_u[dbc.free_dofs]

3987-element Array{Float64,1}:
 -1316.51     
    -6.39355e6
  1148.17     
 -1316.51     
    -6.39355e6
  1148.17     
 -1316.51     
    -6.39355e6
  1148.17     
 -1316.51     
    -6.39355e6
  1148.17     
 -1316.51     
     ⋮        
 -1316.51     
    -6.39355e6
  1148.17     
 -1316.51     
    -6.39355e6
  1148.17     
 -1316.51     
    -6.39355e6
  1148.17     
 -1316.51     
    -6.39355e6
  1148.17     

In [60]:
function check{dim}(cellvalues::CellVectorValues{dim}, 
                    dh::DofHandler, E::SymmetricTensor{4, dim},
                    u::Vector, ɛ_list, σ_list)

    global_dofs = zeros(Int, ndofs_per_cell(dh))
    ɛ = zero(SymmetricTensor{2, dim})
    σ = zero(SymmetricTensor{2, dim})
    Ω = 0.0
    for (cellcount, cell) in enumerate(CellIterator(dh))      
        reinit!(cellvalues, cell)
        celldofs!(global_dofs, cell)
        uu = u[global_dofs]
        for q_point in 1:getnquadpoints(cellvalues)
            dΩ = getdetJdV(cellvalues, q_point)
            ɛ_qp = symmetric(function_gradient(cellvalues, q_point, uu))
            σ_qp = E ⊡ ɛ_qp
            ɛ_list[cellcount] = ɛ_qp
            σ_list[cellcount] = σ_qp
            ɛ += ɛ_qp * dΩ
            σ += σ_qp * dΩ
            Ω += dΩ
        end
    end
    println(Ω)
    return ɛ, σ
end

check (generic function with 1 method)

In [61]:
ɛ_list = fill(zero(SymmetricTensor{2,3}), getncells(grid));
σ_list = fill(zero(SymmetricTensor{2,3}), getncells(grid));
ɛ_box, σ_box = check(cellvalues, dh, E, uu, ɛ_list, σ_list)
display(ɛ_box)
display(σ_box)

3×3 Tensors.SymmetricTensor{2,3,Float64,6}:
  2.02903e-6   0.398878    -8.34501e-7
  0.398878     2.02903e-6  -8.34501e-7
 -8.34501e-7  -8.34501e-7  -3.00105e-6

3×3 Tensors.SymmetricTensor{2,3,Float64,6}:
  4.34121e5    6.13658e10  -1.28385e5
  6.13658e10   4.34121e5   -1.28385e5
 -1.28385e5   -1.28385e5   -3.39738e5

8.00000000000276


In [26]:
σf = reinterpret(Float64, σ_list, (6, getncells(grid)));
ɛf = reinterpret(Float64, ɛ_list, (6, getncells(grid)))


6×5000 Array{Float64,2}:
 -0.000531975   0.00663498   0.00398997  …  -0.00884784   -0.0029498  
  0.0423699     0.0495368    0.0468918       0.0549352     0.0549352  
  0.00205596   -2.93551e-5   0.00189942     -0.00314887   -0.000290439
 -0.000531975   0.00663498   0.00398997     -0.0029498    -0.00884784 
  0.00205596   -2.93551e-5   0.00189942     -0.000290439  -0.00314887 
  0.00112563   -0.00249688  -0.00370941  …   0.00396113    0.00396113 

In [29]:
# Save file
vtkfile = vtk_grid("homo", dh, uu)
vtk_cell_data(vtkfile, ɛf, "strain")
vtk_cell_data(vtkfile, σf, "stress")
vtk_save(vtkfile)

1-element Array{String,1}:
 "homo.vtu"

In [23]:
# Unknowns σ_bar, ɛ_bar
function assemble!{dim}(cellvalues::CellVectorValues{dim}, 
                        σ_bar, ɛ_bar, u,  grid::Grid, dh::DofHandler, E::SymmetricTensor{4, dim})

    # Global matrices and residuals
    residual_u = zeros(ndofs(dh))
    residual_C = zeros(ndofs(dh), 9)
    assembler = start_assemble(K, residual_u)
    
    # Local matrices and residual
    n_basefuncs = getnbasefunctions(cellvalues)
    global_dofs = zeros(Int, ndofs_per_cell(dh))

    residual_ue = zeros(n_basefuncs) # Local stiffness mastrix
    residual_Ce = zeros(n_basefuncs, 9)
    
    Ω = 8

    δɛ = [zero(SymmetricTensor{2, dim}) for i in 1:n_basefuncs]
    ∇δu = [zero(Tensor{2, dim}) for i in 1:n_basefuncs]
    for cell in CellIterator(dh)
        reinit!(cellvalues, cell)
        celldofs!(global_dofs, cell)
        uu = u[global_dofs]
        uuv = reinterpret(Vec{3,Float64}, uu, (4,))
        for q_point in 1:getnquadpoints(cellvalues)
            for i in 1:n_basefuncs
                δɛ[i] = symmetric(shape_gradient(cellvalues, q_point, i)) 
                ∇δu[i] = shape_gradient(cellvalues, q_point, i)
            end
            
            ɛ_qp = symmetric(function_gradient(cellvalues, q_point, uuv))
            σ_qp = E ⊡ ɛ_qp
            dΩ = getdetJdV(cellvalues, q_point)
            for i in 1:n_basefuncs
                Ce[i, :] += -1 / Ω * tovoigt(0.5*(∇u[i] + ∇u[i]) * dΩ)
                residual_ue[i] += 1/Ω * (σ_qp ⊡ ɛ[i] - σ_bar ⊡ (ɛ[i] - ɛ_bar)) * dΩ
                
                residual_Ce[i, :] .+= tovoigt(-1/Ω * (ɛ_qp - ɛ_bar) * dΩ)
            end
        end
        for (i, dof) in enumerate(global_dofs)
            residual_C[dof, :] += residual_Ce[i, :]
        end
        assemble!(residual_u, residual_ue, global_dofs)
    end
    return residual_u, residual_C
end

LoadError: [1m[31merror in method definition: function JuAFEM.assemble! must be explicitly imported to be extended[0m